In [124]:
import pandas as pd
import numpy as np
import math

EX4

In [2]:
df = pd.read_table('../exe_data/benchmark.txt').iloc[9:-2,:]

In [3]:
df.head(3)

,start
9,Benchmarking Training float precision type mna...
10,mnasnet0_5 model average train time : 28.527...
11,Benchmarking Training float precision type mna...


In [4]:
df1 = df[df.index%2 == 1].reset_index().drop('index', axis = 1).rename(columns = {'start':'a'}) # 奇数行
df2 = df[df.index%2 == 0].reset_index().drop('index', axis = 1).rename(columns = {'start':'b'}) # 偶数行
df = pd.concat([df1, df2], axis = 1).astype('string') # 拼接奇数行和偶数行

In [5]:
df.head(3)

,a,b
0,Benchmarking Training float precision type mna...,mnasnet0_5 model average train time : 28.527...
1,Benchmarking Training float precision type mna...,mnasnet0_75 model average train time : 34.10...
2,Benchmarking Training float precision type mna...,mnasnet1_0 model average train time : 34.313...


In [6]:
df['模型'] = df.a.str.extract(r'type ([\w]+)')

In [7]:
df['状态_精度'] = df.a.str.extract(r'Benchmarking ([\w]+) ')+'_'+ df.a.str.extract(r'([\w]+) precision')
df['状态_精度'] = df['状态_精度'].str.replace('ing', '')

In [8]:
df['耗时'] = df.b.str.extract(r'([.\w]+) ms')
df['耗时'] = df['耗时'].astype('float').round(3)

In [9]:
df.drop(['a', 'b'], axis = 1, inplace = True)

In [10]:
df.head(3)

,模型,状态_精度,耗时
0,mnasnet0_5,Train_float,28.528
1,mnasnet0_75,Train_float,34.105
2,mnasnet1_0,Train_float,34.314


In [11]:
col = df.pivot(index = '模型', columns = '状态_精度', values = '耗时').sort_index().columns[::-1]
table = df.pivot(index = '模型', columns = '状态_精度', values = '耗时').sort_index()[col]

In [12]:
table.head(3)

状态_精度,Train_half,Train_float,Train_double,Inference_half,Inference_float,Inference_double
模型,,,,,,
densenet121,88.976,93.357,417.207,19.772,15.637,144.111
densenet161,144.319,136.624,1290.287,27.555,31.750,511.177
densenet169,121.556,104.840,511.404,26.371,21.598,175.808


In [13]:
table.columns.name = ''
table.index.name = ''

In [14]:
table.head(3)

,Train_half,Train_float,Train_double,Inference_half,Inference_float,Inference_double
,,,,,,
densenet121,88.976,93.357,417.207,19.772,15.637,144.111
densenet161,144.319,136.624,1290.287,27.555,31.750,511.177
densenet169,121.556,104.840,511.404,26.371,21.598,175.808


EX5

In [63]:
df1 = pd.read_csv('../exe_data/yali18.csv')
df2 = pd.read_csv('../exe_data/yali19.csv')
df3 = pd.read_csv('../exe_data/qx1819.csv')

In [64]:
df1.head(3)

,Time,MeasName,H0,H1,H2,H3,H4,H5,H6,H7,...,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23
0,2018-01-01,站点4,0.402750,0.407625,0.418125,0.425250,0.426000,0.425250,0.417375,0.426375,...,0.348750,0.359250,0.355500,0.344250,0.352125,0.356250,0.34725,0.343875,0.356625,0.418875
1,2018-01-01,站点7,0.214375,0.226750,0.232375,0.233125,0.235000,0.232750,0.230875,0.220000,...,0.187375,0.196750,0.199750,0.192250,0.186250,0.183250,0.17725,0.163375,0.165250,0.199375
2,2018-01-01,站点22,0.247000,0.248125,0.271375,0.251125,0.272125,0.256375,0.257125,0.242500,...,0.245500,0.242875,0.238375,0.230875,0.237250,0.236875,0.23650,0.236500,0.241000,0.254500


通过df1和df2构造df，把时间设为索引，第一列为站点编号，第二列为对应时刻的压力大小

In [65]:
# 宽表变长表
df = pd.concat([df1, df2]).melt(id_vars = ['Time', 'MeasName'],value_vars = ['H0', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7',
       'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17',
       'H18', 'H19', 'H20', 'H21', 'H22', 'H23'], var_name = ['Hour'], value_name = '压力')

In [66]:
df.head(3)

,Time,MeasName,Hour,压力
0,2018-01-01,站点4,H0,0.402750
1,2018-01-01,站点7,H0,0.214375
2,2018-01-01,站点22,H0,0.247000


In [67]:
# 将MeasName的数字取出，设为'站点'列
df['站点'] = df.MeasName.astype('string').str.replace('站点', '')

In [68]:
# 将时间转化为timedelta后与日期相加
df['t'] = pd.to_datetime(df.Time)+pd.to_timedelta(df.Hour.astype('string').str.replace('H', '').astype('int'), 'H')

In [69]:
# 删掉无用的列
df.drop(columns = ['Time', 'Hour', 'MeasName'], inplace = True)

In [70]:
# 将时间设置为索引，同时修改索引名
df = df.sort_values(by = ['t', '站点']).set_index('t')[['站点', '压力']]
df.index.name = ''

In [71]:
df.head(3)

,站点,压力
,,
2018-01-01,1,0.288625
2018-01-01,10,0.271875
2018-01-01,11,0.260625


In [72]:
df.tail(3)

,站点,压力
,,
2019-12-31 23:00:00,7,0.269875
2019-12-31 23:00:00,8,0.362625
2019-12-31 23:00:00,9,0.201375


在上一问构造的df基础上，构造下面的特征序列或DataFrame，并把它们逐个拼接到df的右侧

    当天最高温、最低温和它们的温差
    当天是否有沙暴、是否有雾、是否有雨、是否有雪、是否为晴天
    选择一种合适的方法度量雨量/下雪量的大小（构造两个序列分别表示二者大小）
    限制只用4列，对风向进行0-1编码

In [73]:
df3['当日最高温'] = df3.气温.astype('string').str.extract('([\d]+)[℃C]').astype('int')
df3['当日最低温'] = df3.气温.astype('string').str.extract('[～ ]([-\d]+)').astype('int')
df3['温差'] = df3['当日最高温'] - df3['当日最低温']

In [82]:
df3['是否有沙暴'] = df3.天气.astype('string').str.contains('沙')
df3['是否有雾'] = df3.天气.astype('string').str.contains('雾')
df3['是否有雨'] = df3.天气.astype('string').str.contains('雨')
df3['是否有雪'] = df3.天气.astype('string').str.contains('雪')
df3['是否为晴天'] = df3.天气.astype('string') == '晴' # 认为只有天气为‘晴’时，才认为是晴天

In [83]:
df3.head(3)

,日期,天气,气温,风向,当日最高温,当日最低温,温差,是否有沙暴,是否有雾,是否有雨,是否有雪,是否为晴天
0,2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5,False,False,False,False,False
1,2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8,False,False,False,False,False
2,2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2,False,False,False,True,False


In [84]:
# 雨 ： 小雨/雨夹雪/雨，小到中雨, 中雨，雷阵雨，中到大雨, 大雨，暴雨 给分1-5， 其他给0分 对于一天内出现多个级别的雨的取平均
# 雪 ： 雨夹雪/小雪，小到中雪，中雪，中到大雪，大雪分别给分1, 1.5, 2，2.5，3, 求平均

In [85]:
rain = {}
rain['小雨'] = 1
rain['雨夹雪'] = 1
rain['雨'] = 1
rain['小到中雨'] = 1.5
rain['中雨'] = 2
rain['雷阵雨'] = 2
rain['中到大雨'] = 2.5
rain['大雨'] = 3
rain['暴雨'] = 4

snow = {}
snow['雨夹雪'] = 1
snow['小雪'] = 1
snow['小到中雪'] = 1.5
snow['中雪'] = 2
snow['中到大雪'] = 2.5
snow['大雪'] = 3

In [86]:
def foo_rain(text):
    a = 0
    l = str.split(text, '转')
    for i in l:
        if i in rain.keys():
            a+=rain[i]
    return round(a/len(l), 2)

def foo_snow(text):
    a = 0
    l = str.split(text, '转')
    for i in l:
        if i in snow.keys():
            a+=snow[i]
    return round(a/len(l), 2)

In [87]:
df3['降雨量'] = df3.天气.astype('string').apply(foo_rain)
df3['降雪量'] = df3.天气.astype('string').apply(foo_snow)

In [88]:
df3['东'] = df3.风向.astype('string').str.contains('东').astype('int')
df3['南'] = df3.风向.astype('string').str.contains('南').astype('int')
df3['西'] = df3.风向.astype('string').str.contains('西').astype('int')
df3['北'] = df3.风向.astype('string').str.contains('北').astype('int')

In [90]:
df['date'] = pd.Series(df.index, index = df.index).dt.date.astype('string')
df3.日期 = df3.日期.astype('string')

In [93]:
df4 = df.merge(df3.drop(columns = ['天气', '气温', '风向']), how = 'left', left_on = 'date', right_on = '日期').drop(columns = '日期')

In [99]:
df4.index = df.index
df = df4
df.dropna(inplace = True)

In [104]:
df.head()

,站点,压力,date,当日最高温,当日最低温,温差,是否有沙暴,是否有雾,是否有雨,是否有雪,是否为晴天,降雨量,降雪量,东,南,西,北
,,,,,,,,,,,,,,,,,
2018-01-01,1,0.288625,2018-01-01,1.0,-4.0,5.0,False,False,False,False,False,0.0,0.0,1.0,1.0,0.0,0.0
2018-01-01,10,0.271875,2018-01-01,1.0,-4.0,5.0,False,False,False,False,False,0.0,0.0,1.0,1.0,0.0,0.0
2018-01-01,11,0.260625,2018-01-01,1.0,-4.0,5.0,False,False,False,False,False,0.0,0.0,1.0,1.0,0.0,0.0
2018-01-01,12,0.280875,2018-01-01,1.0,-4.0,5.0,False,False,False,False,False,0.0,0.0,1.0,1.0,0.0,0.0
2018-01-01,13,0.292500,2018-01-01,1.0,-4.0,5.0,False,False,False,False,False,0.0,0.0,1.0,1.0,0.0,0.0


对df的水压一列构造如下时序特征：

    当前时刻该站点水压与本月的相同整点时间该站点水压均值的差，例如当前时刻为2018-05-20 17:00:00，那么对应需要减去的值为当前月所有17:00:00时间点水压值的均值
    当前时刻所在周的周末该站点水压均值与工作日水压均值之差
    当前时刻向前7日内，该站点水压的均值、标准差、0.95分位数、下雨天数与下雪天数的总和
    当前时刻向前7日内，该站点同一整点时间水压的均值、标准差、0.95分位数
    当前时刻所在日的该站点水压最高值与最低值出现时刻的时间差

In [105]:
df['year'] = pd.to_datetime(df.date).dt.year
df['month'] = pd.to_datetime(df.date).dt.month
df['day'] = pd.to_datetime(df.date).dt.day
df['hour'] = pd.Series(df.index, index = df.index).dt.hour

In [106]:
df.columns

Index(['站点', '压力', 'date', '当日最高温', '当日最低温', '温差', '是否有沙暴', '是否有雾', '是否有雨',
       '是否有雪', '是否为晴天', '降雨量', '降雪量', '东', '南', '西', '北', 'year', 'month',
       'day', 'hour'],
      dtype='object')

In [107]:
g1 = df.pivot(index = ['year', 'month','day', 'hour'], columns = '站点', values = '压力').groupby(['year', 'month', 'hour'])

In [108]:
g1.transform(lambda x: (x-x.mean())).head(3)

站点                          1        10        11        12        13  \
year month day hour                                                     
2018 1     1   0     0.015125 -0.004512 -0.006125  0.008475  0.010713   
               1     0.013200 -0.004325 -0.017350  0.010500  0.007675   
               2     0.009388  0.002725  0.003237  0.009463  0.010713   

站点                         14        15        16   17        18  ...  \
year month day hour                                               ...   
2018 1     1   0     0.013475  0.002200  0.003800  0.0  0.013563  ...   
               1     0.004013  0.001688  0.001000  0.0  0.015062  ...   
               2     0.009700  0.000750 -0.000825  0.0  0.013175  ...   

站点                         28        29         3        30         4  \
year month day hour                                                     
2018 1     1   0     0.001625 -0.002637  0.000688 -0.007737 -0.015275   
               1     0.005850  0.001538  0.001162 -0.014413 -0.011300   
               2    -0.000438  0.001687 -0.001363  0.016375 -0.005500   

站点                          5         6         7         8         9  
year month day hour                                                    
2018 1     1   0     0.001700 -0.012025  0.006088 -0.015146 -0.027862  
               1     0.002662 -0.007150  0.009875 -0.013552 -0.049075  
               2     0.000712 -0.007013  0.011875 -0.008844 -0.038025  

[3 rows x 30 columns]

In [109]:
df.date = pd.to_datetime(df.date)

In [110]:
df_5 = df[df.date.dt.dayofweek.isin([0, 1, 2, 3, 4])].resample('7D')['压力'].mean()
df_2 = df[df.date.dt.dayofweek.isin([5, 6])].resample('7D')['压力'].mean()

In [111]:
tmp = pd.Series(df_5.reset_index()['压力']-df_2.reset_index()['压力'])
tmp.index = pd.Series(df_5.index)
tmp.head(3)


2018-01-01    0.010990
2018-01-08    0.004373
2018-01-15    0.002190
Name: 压力, dtype: float64

In [112]:
r = df.pivot(columns = '站点', values = '压力').rolling('7D')
# 平均值
r.mean().head(3)

站点,1,10,11,12,13,14,15,16,17,18,...,28,29,3,30,4,5,6,7,8,9
,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,0.288625,0.271875,0.260625,0.280875,0.292500,0.261000,0.305250,0.208750,-0.25,0.282750,...,0.325000,0.289125,0.301375,0.250125,0.402750,0.314625,0.249375,0.214375,0.414687,0.255000
2018-01-01 01:00:00,0.290313,0.276750,0.259125,0.284438,0.293813,0.258188,0.309188,0.207625,-0.25,0.286688,...,0.327625,0.290813,0.302313,0.254063,0.405188,0.316500,0.255562,0.220563,0.415937,0.251063
2018-01-01 02:00:00,0.290375,0.282750,0.268500,0.286125,0.296125,0.259125,0.314125,0.206875,-0.25,0.288500,...,0.326500,0.291625,0.302250,0.266125,0.409500,0.317000,0.260750,0.224500,0.419583,0.259250


In [113]:
# 标准差
r.std().tail(3)

站点,1,10,11,12,13,14,15,16,17,18,...,28,29,3,30,4,5,6,7,8,9
,,,,,,,,,,,,,,,,,,,,,
2019-12-31 21:00:00,0.018900,0.056141,0.042338,0.033241,0.062816,0.020671,0.048334,0.018043,0.033950,0.035698,...,0.012672,0.018942,0.017994,0.027119,0.040864,0.018164,0.0,0.028667,0.040922,0.059871
2019-12-31 22:00:00,0.018893,0.056036,0.042331,0.033211,0.063067,0.020665,0.048345,0.018041,0.033947,0.035700,...,0.012662,0.018934,0.017991,0.027074,0.040869,0.018160,0.0,0.028675,0.040926,0.059799
2019-12-31 23:00:00,0.018868,0.056050,0.042360,0.033198,0.063168,0.020657,0.048380,0.017994,0.033940,0.035683,...,0.012657,0.018907,0.017960,0.027062,0.040921,0.018164,0.0,0.028623,0.040968,0.059802


In [114]:
# 0.95分位数
r.quantile(0.95).head(3)

站点,1,10,11,12,13,14,15,16,17,18,...,28,29,3,30,4,5,6,7,8,9
,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,0.288625,0.271875,0.260625,0.280875,0.292500,0.261000,0.305250,0.208750,-0.25,0.282750,...,0.325000,0.289125,0.301375,0.250125,0.402750,0.314625,0.249375,0.214375,0.414687,0.255000
2018-01-01 01:00:00,0.291831,0.281138,0.260475,0.287644,0.294994,0.260719,0.312731,0.208638,-0.25,0.290231,...,0.329988,0.292331,0.303156,0.257606,0.407381,0.318187,0.261131,0.226131,0.417062,0.254606
2018-01-01 02:00:00,0.291850,0.293438,0.284588,0.289350,0.300188,0.261000,0.322913,0.208525,-0.25,0.291975,...,0.329725,0.293175,0.303138,0.287025,0.417075,0.318337,0.270187,0.231813,0.425906,0.273563


In [115]:
df.columns

Index(['站点', '压力', 'date', '当日最高温', '当日最低温', '温差', '是否有沙暴', '是否有雾', '是否有雨',
       '是否有雪', '是否为晴天', '降雨量', '降雪量', '东', '南', '西', '北', 'year', 'month',
       'day', 'hour'],
      dtype='object')

In [187]:
# 下雨天总数
df.pivot(columns = '站点', values = '是否有雨').rolling('7D').sum().apply(lambda x: -(-x//24)).tail()

站点,1,10,11,12,13,14,15,16,17,18,...,28,29,3,30,4,5,6,7,8,9
,,,,,,,,,,,,,,,,,,,,,
2019-12-31 19:00:00,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2019-12-31 20:00:00,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2019-12-31 21:00:00,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2019-12-31 22:00:00,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2019-12-31 23:00:00,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [189]:
# 下雪天总数
df.pivot(columns = '站点', values = '是否有雪').rolling('7D').sum().apply(lambda x: -(-x//24)).tail()

站点,1,10,11,12,13,14,15,16,17,18,...,28,29,3,30,4,5,6,7,8,9
,,,,,,,,,,,,,,,,,,,,,
2019-12-31 19:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-12-31 20:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-12-31 21:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-12-31 22:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-12-31 23:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [190]:
r2 = df.pivot(index = ['date'], columns = ['hour', '站点'], values = '压力').rolling('7D')

In [191]:
# 均值
r2.mean().tail(3)

hour              0                                                     \
站点                 1        10        11        12        13        14   
date                                                                     
2019-12-29  0.315036  0.276000  0.242250  0.267839  0.291696  0.256554   
2019-12-30  0.315786  0.276857  0.244875  0.268000  0.291857  0.256607   
2019-12-31  0.316964  0.274714  0.241179  0.269821  0.292232  0.256071   

hour                                                ...        23            \
站点                15        16        17        18  ...        28        29   
date                                                ...                       
2019-12-29  0.412982  0.217804  0.341089  0.241339  ...  0.304304  0.280929   
2019-12-30  0.413679  0.215607  0.334768  0.241018  ...  0.303393  0.280018   
2019-12-31  0.413786  0.216786  0.335625  0.243214  ...  0.302107  0.277125   

hour                                                                         \
站点                 3        30         4         5    6         7         8   
date                                                                          
2019-12-29  0.318018  0.284839  0.392036  0.265393  0.0  0.291571  0.395464   
2019-12-30  0.317857  0.284679  0.389786  0.264750  0.0  0.291625  0.392946   
2019-12-31  0.315125  0.282911  0.385446  0.264161  0.0  0.287179  0.389089   

hour                  
站点                 9  
date                  
2019-12-29  0.211768  
2019-12-30  0.212411  
2019-12-31  0.211714  

[3 rows x 720 columns]

In [192]:
# 标准差
r2.std().tail(3)

hour              0                                                     \
站点                 1        10        11        12        13        14   
date                                                                     
2019-12-29  0.005123  0.012278  0.011807  0.006444  0.008428  0.005449   
2019-12-30  0.005223  0.012120  0.012000  0.006430  0.008581  0.005523   
2019-12-31  0.006378  0.008303  0.010346  0.008517  0.008613  0.005505   

hour                                                ...        23            \
站点                15        16        17        18  ...        28        29   
date                                                ...                       
2019-12-29  0.023479  0.010061  0.003896  0.010591  ...  0.004920  0.006889   
2019-12-30  0.023625  0.011210  0.018048  0.010632  ...  0.003317  0.005280   
2019-12-31  0.023718  0.012805  0.018643  0.013128  ...  0.004717  0.006726   

hour                                                                         \
站点                 3        30         4         5    6         7         8   
date                                                                          
2019-12-29  0.006322  0.007421  0.012578  0.005249  0.0  0.009232  0.013269   
2019-12-30  0.006061  0.007169  0.010097  0.004362  0.0  0.009231  0.010768   
2019-12-31  0.007002  0.007916  0.016418  0.005058  0.0  0.011240  0.015811   

hour                  
站点                 9  
date                  
2019-12-29  0.009449  
2019-12-30  0.010288  
2019-12-31  0.010920  

[3 rows x 720 columns]

In [193]:
# 0.95分位数
r2.quantile(0.95).tail(3)

hour              0                                                     \
站点                 1        10        11        12        13        14   
date                                                                     
2019-12-29  0.321588  0.294263  0.257888  0.273813  0.300225  0.262538   
2019-12-30  0.322038  0.294263  0.258225  0.273813  0.300562  0.262800   
2019-12-31  0.324812  0.283762  0.253163  0.280150  0.300562  0.262800   

hour                                              ...        23            \
站点                15        16      17        18  ...        28        29   
date                                              ...                       
2019-12-29  0.432450  0.226525  0.3447  0.247688  ...  0.311000  0.290850   
2019-12-30  0.432450  0.226525  0.3447  0.247688  ...  0.306538  0.286388   
2019-12-31  0.432675  0.232000  0.3480  0.257438  ...  0.306538  0.285038   

hour                                                                         \
站点                 3        30         4         5    6         7         8   
date                                                                          
2019-12-29  0.326225  0.293550  0.408600  0.272213  0.0  0.302575  0.412575   
2019-12-30  0.325438  0.292763  0.402675  0.269812  0.0  0.302575  0.406950   
2019-12-31  0.324650  0.292763  0.402675  0.269812  0.0  0.301225  0.406950   

hour                  
站点                 9  
date                  
2019-12-29  0.222562  
2019-12-30  0.224700  
2019-12-31  0.224700  

[3 rows x 720 columns]

In [194]:
# 当前时刻所在日的该站点水压最高值与最低值出现时刻的时间差
df.groupby(['date','站点'])['压力'].idxmax() - df.groupby(['date','站点'])['压力'].idxmin()

date        站点
2018-01-01  1    -1 days +18:00:00
            10   -1 days +18:00:00
            11   -1 days +16:00:00
            12   -1 days +18:00:00
            13   -1 days +18:00:00
                        ...       
2019-12-31  5    -1 days +08:00:00
            6      0 days 00:00:00
            7    -1 days +05:00:00
            8    -1 days +08:00:00
            9    -1 days +17:00:00
Name: 压力, Length: 21870, dtype: timedelta64[ns]